# About this notebook

This notebook uses PyCaret for model implementation,

This notebook does the following...
- Get the top forecasting models for each climate variable
- Create forecasts on each climate variable
- Find better forecasts with variation for flat / straight line forecasts
- Create 6-year forecast, combine, and output `climate_forecast.csv`


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.regression import *
from pycaret.time_series import *

In [2]:
climate_df = pd.read_csv('./climateipynb_output.csv', names=['Date', 'Rainfall', 'Max Temperature', 'Min Temperature', 'Relative Humidity', 'Wind Speed'], header=0)

rice_df = pd.read_csv('./riceipynb_output1.csv', names=['Date', 'Irrigated Palay Production', 'Rainfed Palay Production'], header=0)

In [ ]:
climate_rice_df = pd.merge(climate_df, rice_df,  
                   on='Date',  
                   how='outer')

climate_rice_df.info() 

In [ ]:
climate_series = climate_df.copy()
climate_series.index = pd.date_range(start='2013-03', end='2023', freq='3M')
climate_series.head()

In [5]:
s_RF = TSForecastingExperiment()
s_TMAX = TSForecastingExperiment()
s_TMIN = TSForecastingExperiment()
s_RH = TSForecastingExperiment()
s_WS = TSForecastingExperiment()

In [ ]:
s_RF.setup(data = climate_series[['Rainfall']], session_id = 321, verbose = False)
s_TMAX.setup(data = climate_series[['Max Temperature']], session_id = 321, verbose = False)
s_TMIN.setup(data = climate_series[['Min Temperature']], session_id = 321, verbose = False)
s_RH.setup(data = climate_series[['Relative Humidity']], session_id = 321, verbose = False)
s_WS.setup(data = climate_series[['Wind Speed']], session_id = 321, verbose = False)


In [ ]:
best_RF = s_RF.compare_models()
best_TMAX = s_TMAX.compare_models(n_select=10)
best_TMIN = s_TMIN.compare_models()
best_RH = s_RH.compare_models()
best_WS = s_WS.compare_models(n_select=30)

In [ ]:
print(best_RF)
print(best_TMAX[3])
print(best_TMIN)
print(best_RH)
print(best_WS[-3])


In [ ]:
s_WS.plot_model(best_WS, plot = 'forecast', data_kwargs = {'fh' : 24})

In [ ]:
s_RF.plot_model(best_RF, plot = 'forecast', data_kwargs = {'fh' : 24})

In [ ]:
s_RF.plot_model(best_RF, plot = 'forecast', data_kwargs = {'fh' : 24})
s_TMAX.plot_model(best_TMAX[3], plot = 'forecast', data_kwargs = {'fh' : 24})
s_TMIN.plot_model(best_TMIN, plot = 'forecast', data_kwargs = {'fh' : 24})
s_RH.plot_model(best_RH, plot = 'forecast', data_kwargs = {'fh' : 24})
s_WS.plot_model(best_WS[-3], plot = 'forecast', data_kwargs = {'fh' : 24})

In [12]:
climate_forecast = pd.DataFrame()

In [13]:
RF_predictions = s_RF.predict_model(best_RF, fh = 24, verbose = False)
climate_forecast = climate_forecast.assign(Rainfall=RF_predictions)
TMAX_predictions = s_TMAX.predict_model(best_TMAX[3], verbose = False, fh = 24)
climate_forecast = climate_forecast.assign(Max_Temperature=TMAX_predictions)
TMIN_predictions = s_TMIN.predict_model(best_TMIN, verbose = False, fh = 24)
climate_forecast = climate_forecast.assign(Min_Temperature=TMIN_predictions)
RH_predictions = s_RH.predict_model(best_RH, verbose = False, fh = 24)
climate_forecast = climate_forecast.assign(Relative_Humidity=RH_predictions)
WS_predictions = s_WS.predict_model(best_WS[-3], verbose = False, fh = 24)
climate_forecast = climate_forecast.assign(Wind_Speed=WS_predictions)

In [ ]:
climate_forecast.head()

In [15]:
climate_forecast.to_csv('climateforecast.csv')